# Topic Modelling for Information Tech M&A Deal Ratonale's

In [4]:
import re
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from stop_words import safe_get_stop_words
from stop_words import get_stop_words
from textblob import TextBlob

In [5]:
it_comp = pd.read_excel("D:\\ISB\\Capstone Project\\Data\\4-IT Deals\\4-Information Technology_20171016.xls",sheetname='MM_DealUnformattedReport')#,skiprows=7)
for i in range(0,len(it_comp)):
    try:
        it_comp['Deal Description'][i] = it_comp['Deal Description'][i].encode('utf8')
    except:
        print i

C:\Users\Rajiv2806\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
stop_words1 = set(stopwords.words('english'))
stop_words1 = " ".join(i.encode('utf-8') for i in stop_words1).split()
stop_words2 = ['million','approximately','july','march','august','february','october','press','release','usbased','usd'
               ,'company','form','filed','undisclosed']
stop_words3 = list(get_stop_words('en'))
stop_words3 = " ".join(i.encode('utf-8') for i in stop_words3).split()
stop_words4 = list(safe_get_stop_words('unsupported language'))
stop_words5 = str(it_comp[['Target Company','Bidder Company']]) #,'Sellers','Target Ticker','Exchange:Ticker','Deal Responses','Target Company Industry']])
stop_words5 = list(pd.unique(stop_words5.split()))
stop_words = list(pd.unique(stop_words1+stop_words2+stop_words3+stop_words4+stop_words5))

stemming1 = WordNetLemmatizer()    #stemming2 = PorterStemmer()

In [10]:
unigram = []
bigram = []

for i in range(0,len(it_comp)):
    to_lowercase = it_comp['Deal Description'][i].lower()
    puncuation_removed = re.sub(r'[^\w\s]','',to_lowercase)
    newline_removed = re.sub('\n',' ',puncuation_removed)
    stopwords_removed = " ".join(j for j in newline_removed.split() if j not in stop_words)
    stemming_words1 = " ".join(stemming1.lemmatize(j) for j in stopwords_removed.split())
    numbers_removed = " ".join(re.sub("\d+","",i) for i in stemming_words1.split())
    letters_removed = " ".join(j for j in numbers_removed.split() if len(j) > 3)
    trimmed = letters_removed.strip()
    unigram.append(trimmed)
    
    a = ""
    for j in range(0,(len(trimmed.split())-1)):
        empty = trimmed.split()[j]+"_"+trimmed.split()[j+1]
        a = a+" "+empty
    a = a.strip()
    bigram.append(a)
    
data = [str(bigram[i])+" "+str(unigram[i]) for i in range(len(bigram))]

n_features = 1000

# tf or count matrix
# for topic modelling LDA can only use raw term counts because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=8, max_features=n_features, stop_words='english')
tf = tf_vectorizer.fit_transform(data)
tf_feature_names = tf_vectorizer.get_feature_names()

# tf-idf matrix
# for topic modelling NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=8, max_features=n_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(data)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

n_topics = 8 #input("No of topics:")

nmf = NMF(n_components=n_topics,random_state=1,alpha=.1,l1_ratio=.5,init='nndsvd')
nmf = nmf.fit(tfidf)

lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0)
lda = lda.fit(tf)

# print "The Shape of NMF Matrix:",nmf.components_.shape
# print "The Shape of LDA Matrix:",lda.components_.shape

def display_topics(model, feature_names, no_top_words):
    if str(model)[0:3] == "NMF":
        print "NMF MODEL"
    elif str(model)[0:3] == "Lat":
        print "LDA MODEL"
        
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx+1)
        print " ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])
    print "="*100
    
no_top_words = 15

print "Topics For Information Technology"
print "="*100

display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tfidf_feature_names, no_top_words)

Topics For Information Technology
NMF MODEL
Topic 1:
service solution technology acquisition business transaction providing engaged provider security consulting management engaged_providing customer source
Topic 2:
venture capital venture_capital capital_firm firm partner venture_partner funding investor raised private_equity private group venture_venture round
Topic 3:
share headquartered transaction expected price closing corporation termination condition offer exhibit agreement cash acquire approval
Topic 4:
partner equity private private_equity equity_firm capital firm partner_private firm_acquired equity_partner capital_partner portfolio acquired stake management
Topic 5:
medium online mobile social content brand video digital marketing platform acquired game advertising entertainment acquisition
Topic 6:
software software_solution solution management management_software customer enterprise provider acquired software_service acquisition development portfolio enable cloudbased
Topi

In [12]:
topic_output_lda = np.column_stack((data,lda.transform(tf)))
topic_output_nmf = np.column_stack((data,nmf.transform(tfidf)))
df1 = pd.DataFrame(topic_output_lda,columns=['Deal Rationale','Topic1','Topic2','Topic3','Topic4','Topic5','Topic6','Topic7','Topic8'])
df2 = pd.DataFrame(topic_output_nmf,columns=['Deal Rationale','Topic1','Topic2','Topic3','Topic4','Topic5','Topic6','Topic7','Topic8'])
df1["LDA Topic"] = df1[df1.columns[1:9]].idxmax(axis=1)
df2["NMF Topic"] = df2[df2.columns[1:9]].idxmax(axis=1)

it_comp["NMF TopicID"] = df2[df2.columns[1:9]].idxmax(axis=1)
it_comp["LDA TopicID"] = df1[df1.columns[1:9]].idxmax(axis=1)

it_comp.to_csv("D:\\ISB\\Capstone Project\\Data\\4-IT Deals\\Information Tech - Topics.csv",index=False,encoding='utf-8')

## Summary:

Topic modelling on Information Technology acquisitions is done using the LDA and NMF methods.

Results for the the Deal rationale, for 8 topics of both the models are:

**Topics from Deal Description are:**

**NMF MODEL**

Topic 1:service solution technology acquisition business transaction providing engaged provider security consulting management engaged_providing customer source –technology and customer

Topic 2:venture capital venture_capital capital_firm firm partner venture_partner funding investor raised private_equity private group venture_venture round –firm & venture

Topic 3:share headquartered transaction expected price closing corporation termination condition offer exhibit agreement cash acquire approval –deal details

Topic 4:partner equity private private_equity equity_firm capital firm partner_private firm_acquired equity_partner capital_partner portfolio acquired stake management - partnerships

Topic 5:medium online mobile social content brand video digital marketing platform acquired game advertising entertainment acquisition –online marketing

Topic 6:software software_solution solution management management_software customer enterprise provider acquired software_service acquisition development portfolio enable cloudbased –software 

Topic 7:health healthcare care medical clinical patient hospital analytics solution april outcome information practice home management –healthcare tech

Topic 8:data data_center center cloud infrastructure analytics storage managed hosting data_analytics customer enterprise hybrid network managed_service – Data analytics and cloud

**LDA MODEL**

Topic 1:venture capital firm partner venture_capital private equity private_equity capital_firm equity_firm venture_partner group headquartered acquired investor

Topic 2:fuel server event monitoring software software_acquired capital access management acquisition agency transaction management_software customer optimization

Topic 3:service acquisition financial data acquired provider business listed solution center source transaction payment link source_link

Topic 4:share transaction offer price closing termination corporation headquartered expected shareholder agreement deal approval board condition

Topic 5:solution acquired service software acquisition management technology provider source transaction link source_link engaged customer equity

Topic 6:connection virtual venture firm acquired partner capital venture_capital equity senior_vice solution capital_firm private platform acquisition

Topic 7:roper roper_technology industrial equipment innovation product_solution manufacturer technology_listed information_system remain medical headquarters longterm month expects

Topic 8:acquisition transaction service expected technology cash acquire solution update corporation headquartered completed condition business listed
